In [6]:
import os
import numpy as np
import cv2
from PIL import Image
from operator import itemgetter
import tools as tl
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from matplotlib import pylab as plt
from scipy.stats import entropy, norm
import gc

In [7]:
data='melanoma'#or 'melanoma'
method='unet_patch'
model_name="unet"

In [8]:
#Directry を作る関数
def my_makedirs(path):
    if not os.path.isdir(path):
        os.makedirs(path)

In [ ]:
#正規分布に基づいたstep幅を決める関数
sigma=0.4#0~1の間, 0.4がbest
def gaussian_dist(sigma,max_step):
    X = np.arange(0,1,0.01)
    Y = norm.pdf(X,0,sigma)
    h = 1/Y[0]*max_step
    Y = Y*h
    return Y

In [9]:
#label付けする関数. ipsなら赤が1, 緑が2, 青が3のmapにする. melanomaは0か1
#mask_array...mask画像
#data..."ips"or"melanoma"
def get_ytrue(mask_array, data):
        if data == "ips":
            height, width, _ = mask_array.shape
            good_label = ((mask_array[:,:,0] == 255)&
                          (mask_array[:,:,1] == 0)&
                          (mask_array[:,:,2] == 0)
                         ) * np.ones((height, width)) * 1
            bad_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 255)&
                         (mask_array[:,:,2] == 0)
                        ) * np.ones((height, width)) * 2
            bgd_label = ((mask_array[:,:,0] == 0)&
                         (mask_array[:,:,1] == 0)&
                         (mask_array[:,:,2] == 255)
                        ) * np.ones((height, width)) * 3
            y_true = good_label + bad_label + bgd_label
            return y_true
        else:
            height, width = mask_array.shape
            y_true = (mask_array == 255) * np.ones((height, width))
            return y_true

In [10]:
def pre_data_path_load(path):#一つ前のCurriculumでのuncertainty mapをloadする関数
    img_path=[]
    if data in ['ips', 'melanoma']:
        for x in os.listdir(path):
            img_path.append(path+x)
        return img_path

In [13]:
#メラノーマの等間隔でのpatchの切り出し
number=0
step=130
data='melanoma'
patch_size=160
for data_num in [1,2,3,4,5]:
    for w in ['train']:#'train','test'
        m=0
        print(w)
        #画像path読み込み
        img_list, mask_list = tl.data_path_load(data,w,data_num)
        img_list.sort()
        mask_list.sort()
        for i_path, m_path in zip(img_list, mask_list):         
            my_makedirs('/home/sora/new_project/crop/dataset_%d/%s/'%(data_num,w))
            y=0
            if number % 100 ==0:
                print(number)
            number+=1
            #画像読み込み
            m_mask = cv2.imread(m_path)
            m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
            height, width = m_mask.shape[:2]

            img = Image.open(i_path)
            img = np.array(img)

            while y < (height - patch_size + 1):
                x = 0
                while x < (width - patch_size + 1):
                    list=[]
                    crop_mask = m_mask[y:patch_size+y, x:patch_size+x]
                    crop_img = img[y:patch_size+y, x:patch_size+x] 
                    crop_mask = crop_mask/255
                    crop_img = crop_img/255.
                    
                    #教師ラベル作成. 0, 1チャネルをlabelに使う
                    tmp=np.ones((patch_size,patch_size,3))
                    tmp[crop_mask==1]=0#３次元配列のチャネル0に黒い部分を1とした１次元行列を代入
                    tmp[:,:,1]=crop_mask[:,:,0]#３次元配列のチャネル1に白い部分を1とした１次元行列を代入

                    x+=step

                    list.append(crop_img)
                    list.append(tmp)
                    np.save('/home/sora/new_project/crop/dataset_%d/%s/input%d.npy' %(data_num,w,m), np.array(list))
                    m+=1
                    del list,tmp,crop_img,crop_mask
                    gc.collect()
                y+=step

train
0
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


NameError: name 'a' is not defined

In [ ]:
number=0
dist = gaussian_dist(sigma, max_step)#正規分布作成
step=120
data='melanoma'
patch_size=160
for data_num in [3,4,5]:
    for w in ['train']:#'train','test'
        n=0
        m=0
        print(w)
        path=("./%s/%s/%s/size_%d/dataset_%d/%s_un/uncertainty_T_0.5/" %(data, method,model_name,patch_size,data_num,w))
        img_list, mask_list = tl.data_path_load(data,w,data_num)#画像のpathを取得
        pre_list = pre_data_path_load(path)
        img_list.sort()
        mask_list.sort()
        pre_list.sort()
        for i_path, m_path,p_path in zip(img_list, mask_list, pre_list): 
            my_makedirs('/home/sora/new_project/crop/dataset_%d/%s/'%(data_num,w))
            y=0
            if number % 50 == 0:
                print(number)
            number+=1
            m_mask = cv2.imread(m_path)
            m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
            height, width = m_mask.shape[:2]

            img = Image.open(i_path)
            img = np.array(img)

            p_image = cv2.imread(p_path)#uncertainty map load
            p_image = cv2.cvtColor(p_image, cv2.COLOR_BGR2RGB)

            while y < (height - patch_size + 1):
                x = 0
                while x < (width - patch_size + 1):
                    list=[]
                    crop_mask = m_mask[y:patch_size+y, x:patch_size+x]

                    crop_img = img[y:patch_size+y, x:patch_size+x]  
                    crop_p = p_image[y:patch_size+y, x:patch_size+x]

                    crop_mask = crop_mask/255
                    crop_img = crop_img/255.

                    tmp=np.ones((patch_size,patch_size,3))
                    tmp[crop_mask==1]=0
                    tmp[:,:,1]=crop_mask[:,:,1]

                    e = np.count_nonzero(crop_p[:,:,0] == 255)/(patch_size*patch_size)#uncertainty map の白い部分の割合を計算
                    e = round(e, 3)
                    if e == 1:
                        e = 0.999
                    x_step = int(dist[int(e*100)])
                    x+=x_step
                    list.append(crop_img)
                    list.append(tmp)

                    np.save('/home/sora/new_project/crop/dataset_%d/%s/input%d.npy' %(data_num,w,m), np.array(list))
                    m+=1
                    del list,crop_img,crop_mask,crop_p
                    gc.collect()
                y+=step

In [ ]:
#melanoma画像の縦、横の短い方のピクセル幅に合わせてpatchを切り出す
number=0
for w in ['train','val']:#'train','test'
    m=0
    print(w)
    img_list, mask_list = tl.data_path_load(data,w,data_num)
    img_list.sort()
    mask_list.sort()
    for i_path, m_path in zip(img_list, mask_list):         
        #my_makedirs('/home/sora/new_project/crop/%s/inputup'%w)
        my_makedirs('/home/sora/new_project/crop/%s/inputdown'%w)
        y=0
        if number % 100 ==0:
            print(number)
        number+=1
        m_mask = cv2.imread(m_path)
        m_mask = cv2.cvtColor(m_mask, cv2.COLOR_BGR2RGB)
        height, width = m_mask.shape[:2]
        #mask = Image.open(m_path)
        #height, width = m_mask.size
        #mask = np.array(mask)
        if height < width:
            step = width-height
            size = height
        elif height > width:
            step = height-width
            size = width
        else:
            step=10
            size = width
        img = Image.open(i_path)
        img = np.array(img)
         
        while y < (height - size + 1):
            x = 0
            while x < (width - size + 1):
                list=[]
                crop_mask = m_mask[y:size+y, x:size+x]
                crop_img = img[y:size+y, x:size+x] 
                

                
                crop_img = Image.fromarray(crop_img)
                crop_mask = Image.fromarray(crop_mask)
                crop_img = crop_img.resize((patch_size,patch_size))
                crop_mask = crop_mask.resize((patch_size,patch_size))
                
                crop_img = np.array(crop_img)
                crop_mask = np.array(crop_mask)
                crop_mask = crop_mask/255
                crop_img = crop_img/255.
                crop_mask[crop_mask>0]=1
                
                tmp=np.ones((patch_size,patch_size,3))
                tmp[crop_mask==1]=0
                tmp[:,:,1]=crop_mask[:,:,0]
                
                x+=step
                
                list.append(img)
                list.append(tmp)
                np.save('/home/sora/new_project/crop/%s/inputdown/inputdown_%d.npy' %(w,m), np.array(list))
                m+=1
            y+=step
